In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# Hardcode all variables
window_size = 48
batch_size_exp = 1
epoch_exp = 2
neurons_exp = 50
predict_values_exp = 6000
lag_exp=48

In [3]:
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [4]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

In [5]:
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [6]:
def scale(data_norm):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(data_norm)
    # transform train
    data_norm = data_norm.reshape(data_norm.shape[0], data_norm.shape[1])
    data_scaled = scaler.transform(data_norm)
    return scaler, data_scaled

In [7]:
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [8]:
from keras.layers import Activation, Dense, BatchNormalization, TimeDistributed
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons_exp, dropout = 0.1 ,batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(BatchNormalization())
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dense(50))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [9]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=1)
    return yhat[0,0]

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_64c85547f35d43b5b41b68c871da81d6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HcWLNSvsllJ6yiZ0P9bQiEHBvJ5Ty-JY0OoShzpA1FLl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_64c85547f35d43b5b41b68c871da81d6.get_object(Bucket='hanif-donotdelete-pr-nx1leyc1obvpqp',Key='winddirtime.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

series = pd.read_csv(body,index_col='Date/Time')
series.head()


,Wind Direction (°)
Date/Time,
01 01 2018 00:00,259.994904
01 01 2018 00:10,268.641113
01 01 2018 00:20,272.564789
01 01 2018 00:30,271.258087
01 01 2018 00:40,265.674286


In [11]:
raw_values = series.values
diff_values = difference(raw_values, 1)

In [12]:
supervised = timeseries_to_supervised(diff_values, lag_exp)
supervised_values = supervised.values

In [13]:
# split data into train and test-sets
scaler,supervised_values = scale(supervised_values)
train_scaled, test_scaled = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
#print(test_scaled)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
# fit the model
lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
44529/44529 [==============================] - 882s 20ms/step - loss: 0.0131
Epoch 1/1
44529/44529 [==============================] - 877s 20ms/step - loss: 0.0131


In [15]:
# walk-forward validation on the test data
predictions = list()
expectations = list()
for i in range(len(test_scaled)-window_size):
    window_prediction_frame = test_scaled
    test_pred = list()
    for j in range(window_size):
        X, y = window_prediction_frame[i, 0:-1], window_prediction_frame[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)#batch_size_exp to 1
        '''# Start Debug prints
        print("X: %", X)
        print("yhat: %", yhat)
        # End Debug prints'''
        # Replacing value in test scaled with the predicted value.
        test_pred = [yhat] + test_pred 
        if len(test_pred) > lag_exp+1:
            test_pred = test_pred[:-1]
        if j+1<len(window_prediction_frame):
            if j+1 > lag_exp+1:
                window_prediction_frame[j+1] = test_pred
            else:
                window_prediction_frame[j+1] = np.concatenate((test_pred, window_prediction_frame[j+1, j+1:]),axis=0)

    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    expected = raw_values[len(train_scaled) + i + 1 + window_size]
    predictions.append(yhat)
    expectations.append(expected)
    #print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

In [27]:
expectations = np.array(expectations)
predictions = np.array(predictions)
print("Mean Absolute Percent Error: ", (np.mean(np.abs((expectations - predictions) / expectations.mean()))*100))

Mean Absolute Percent Error:  38.13166151444166


In [28]:
lstm_model

In [67]:
import math
testScore = model.evaluate(X, Y, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

NameError: name 'model' is not defined

In [29]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [30]:
wml_credentials = {
  "apikey": "mMtFox3bgn8ZwUo8OxuT_7uegZO6HNwmdj55cde-x4Tg",
  "iam_apikey_description": "Auto-generated for key 22942c5d-09db-432c-aaa8-3769965759b9",
  "iam_apikey_name": "Service credentials-2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/1af47d91ccdc47489f9955bf340a1583::serviceid:ServiceId-a1a64fbd-7e6c-4f1b-9e25-799208ee5c1a",
  "instance_id": "fdc0cfe6-e3a5-4958-af00-d1453d727e98",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [31]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [32]:
metadata = {
        client.repository.ModelMetaNames.AUTHOR_NAME: 'Mohamed Abdullah',
        client.repository.ModelMetaNames.AUTHOR_EMAIL: 'abdullahmohamed2540@gmail.com',
        client.repository.ModelMetaNames.NAME: 'Prediction of Wind Direction',
        client.repository.ModelMetaNames.FRAMEWORK_NAME: 'tensorflow',
        client.repository.ModelMetaNames.FRAMEWORK_VERSION: '1.15',
        client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES:[{'name':'keras', 'version':'2.2.4'}]
}

In [33]:
model_result_path = 'direction_prediction.h5'
lstm_model.save(model_result_path)

In [34]:
!tar -zcvf direction_prediction.tgz direction_prediction.h5

direction_prediction.h5


In [35]:
stored_data = client.repository.store_model(model='direction_prediction.tgz' ,meta_props=metadata, training_data =X, training_target=y)

In [36]:
stored_data

{'metadata': {'guid': 'a1a2b136-4ac0-4a4f-a97d-63d260b076b9',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/fdc0cfe6-e3a5-4958-af00-d1453d727e98/published_models/a1a2b136-4ac0-4a4f-a97d-63d260b076b9',
  'created_at': '2020-07-11T18:53:56.503Z',
  'modified_at': '2020-07-11T18:53:56.556Z'},
 'entity': {'runtime_environment': 'None Provided',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/fdc0cfe6-e3a5-4958-af00-d1453d727e98/published_models/a1a2b136-4ac0-4a4f-a97d-63d260b076b9/learning_configuration',
  'author': {'name': 'Mohamed Abdullah'},
  'name': 'Prediction of Wind Direction',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/fdc0cfe6-e3a5-4958-af00-d1453d727e98/published_models/a1a2b136-4ac0-4a4f-a97d-63d260b076b9/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/fdc0cfe6-e3a5-4958-af00-d1453d727e98/published_models/a1a2b136-4ac0-4a4f-a97d-63d260b076b9/feedback',
  'l

In [37]:
guid = client.repository.get_model_uid(stored_data)

In [38]:
guid

'a1a2b136-4ac0-4a4f-a97d-63d260b076b9'

In [39]:
model_deployment_details = client.deployments.create(guid)



#######################################################################################

Synchronous deployment creation for uid: 'a1a2b136-4ac0-4a4f-a97d-63d260b076b9' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='27b4c695-9fb4-4c22-84a4-c73b2d6d0f86'
------------------------------------------------------------------------------------------------




In [40]:
scoring_endpoint = client.deployments.get_scoring_url(model_deployment_details)

In [41]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/fdc0cfe6-e3a5-4958-af00-d1453d727e98/deployments/27b4c695-9fb4-4c22-84a4-c73b2d6d0f86/online'

In [64]:
x = np.array([[202.45125917],
       [198.3243857 ],
       [200.6060786 ]])
      



In [66]:
lstm_model

In [60]:
y

0.0005912450812423291

In [63]:
predictions

array([[202.45125917],
       [198.3243857 ],
       [200.6060786 ],
       ...,
       [ 80.74093875],
       [ 84.77138973],
       [ 83.76924722]])